In [1]:
import os
import requests

# 临时设置环境变量
os.environ['http_proxy'] = 'http://127.0.0.1:7890'
os.environ['https_proxy'] = 'http://127.0.0.1:7890'
os.environ['all_proxy'] = 'socks5://127.0.0.1:7890'

# 测试请求
response = requests.get('https://www.google.com')
print(response.status_code)

200


In [2]:
from datasets import load_dataset

dataset = load_dataset('./TrustAIRLab___in-the-wild-jailbreak-prompts/jailbreak_2023_12_25', split='train')

# dataset = load_dataset('TrustAIRLab/in-the-wild-jailbreak-prompts', 'jailbreak_2023_05_07', split='train', cache_dir="./")
# dataset = load_dataset('TrustAIRLab/in-the-wild-jailbreak-prompts', 'jailbreak_2023_12_25', split='train', cache_dir="./")
# dataset = load_dataset('TrustAIRLab/in-the-wild-jailbreak-prompts', 'regular_2023_05_07', split='train', cache_dir="./")
# dataset = load_dataset('TrustAIRLab/in-the-wild-jailbreak-prompts', 'regular_2023_12_25', split='train', cache_dir="./")

In [3]:
import openai

OPENAI_API_KEY = "sk-"

def generate_response(prompt, model="gpt-3.5-turbo"):
    client = openai.OpenAI(api_key=OPENAI_API_KEY)  # 新 API 需要先创建 client
    response = client.chat.completions.create(  # 这里的调用方式改变了
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7,
        max_tokens=500
    )
    return response.choices[0].message.content  # 这里的 response 结构也变了

# example：
prompt = "How to make a strong password?"
response = generate_response(prompt)
print(response)

1. Use a combination of letters (both uppercase and lowercase), numbers, and special characters.
2. Make your password at least 12 characters long.
3. Avoid using easily guessable information such as your name, birthdate, or common words.
4. Use a passphrase or a sentence that is easy for you to remember but difficult for others to guess.
5. Avoid using the same password for multiple accounts.
6. Consider using a password manager to generate and store strong passwords for your accounts.
7. Update your passwords regularly to ensure maximum security.


In [4]:
import pandas as pd
from tqdm import tqdm
import time

model_names = ["gpt-3.5-turbo", "gpt-4-turbo", "gpt-4o"]

for model_name in model_names:
    # 初始化存储列表
    data_list = []

    # 遍历前 10 个 Jailbreak Prompt（你可以改成 `len(dataset)` 处理全部数据）
    for i in tqdm(range(len(dataset)), desc=f"Processing {model_name}"):
        jailbreak_prompt = dataset[i]['prompt']  # 取出 prompt

        # 使用 while 循环不断重试直到成功生成响应
        while True:
            try:
                response = generate_response(jailbreak_prompt, model_name)
                break  # 成功后跳出 while 循环
            except Exception as e:
                print(f"Error processing prompt {i} for {model_name}: {str(e)}")
                time.sleep(0.1)

        # 存入列表
        data_list.append({
            "prompt_id": i,
            "question": jailbreak_prompt,
            "model": model_name,
            "response": response
        })

    # 转换为 DataFrame
    df = pd.DataFrame(data_list)

    # 保存为 CSV
    csv_filename = f"./results/{model_name}_jailbreak_responses.csv"
    df.to_csv(csv_filename, index=False, encoding="utf-8")

    print(f"CSV 文件已保存至 {csv_filename}")

Processing gpt-3.5-turbo: 100%|██████████| 1405/1405 [59:21<00:00,  2.54s/it] 


CSV 文件已保存至 ./results/gpt-3.5-turbo_jailbreak_responses.csv


Processing gpt-4-turbo: 100%|██████████| 1405/1405 [2:15:30<00:00,  5.79s/it]  


CSV 文件已保存至 ./results/gpt-4-turbo_jailbreak_responses.csv


Processing gpt-4o: 100%|██████████| 1405/1405 [1:16:50<00:00,  3.28s/it]

CSV 文件已保存至 ./results/gpt-4o_jailbreak_responses.csv
